# Day 1-2: Gmail Connection & Email Fetching

**Goal**: Connect to Gmail API, authenticate, and fetch newsletter emails

**What You'll Learn**:
- Gmail API authentication with OAuth2
- Fetching emails with queries
- Parsing email content
- Understanding email data structures

**Success Metrics**:
- ✅ Successfully authenticate with Gmail
- ✅ Fetch last 7 days of emails
- ✅ Parse email content into readable format

## Step 1: Environment Setup

First, let's import everything we need and set up our environment.

In [ ]:
# Cell 1: Install required packages (run once)
# Uncomment and run if packages not installed
# !pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client python-dotenv

In [ ]:
# Cell 2: Import libraries
import os
import sys
from datetime import datetime, timedelta
import json
from pathlib import Path

# Add src to path so we can import our modules
sys.path.append(str(Path.cwd().parent / 'src'))

# Load environment variables
from dotenv import load_dotenv
load_dotenv(Path.cwd().parent / '.env')

print("✓ Libraries imported successfully")
print(f"Working directory: {Path.cwd()}")
print(f"Python version: {sys.version}")

## Step 2: Gmail Authentication

Let's authenticate with Gmail using OAuth2.

**Before running this**: Make sure you have `credentials.json` in your project root.

Get it from: https://console.cloud.google.com/apis/credentials

In [ ]:
# Cell 3: Import our Gmail authentication module
from email_agent.gmail_auth import GmailAuth, setup_gmail_auth

print("✓ Gmail auth module imported")

In [ ]:
# Cell 4: Set up authentication
# This will:
# 1. Look for existing token
# 2. Refresh if expired
# 3. Or start new auth flow (opens browser)

print("Starting Gmail authentication...")
gmail_auth = setup_gmail_auth()

print("\n✓ Authentication complete!")

## Step 3: Test the Connection

Let's verify we can connect and see what's in your Gmail account.

In [ ]:
# Cell 5: Get Gmail service and test
gmail_service = gmail_auth.get_gmail_service()

# Get profile info
profile = gmail_service.users().getProfile(userId='me').execute()

print("📧 Gmail Account Info:")
print(f"   Email: {profile['emailAddress']}")
print(f"   Total Messages: {profile['messagesTotal']:,}")
print(f"   Total Threads: {profile['threadsTotal']:,}")

# Let's see what labels you have
labels_result = gmail_service.users().labels().list(userId='me').execute()
labels = labels_result.get('labels', [])

print(f"\n📁 Available Labels ({len(labels)}):")
for label in labels[:10]:  # Show first 10
    print(f"   - {label['name']}")
if len(labels) > 10:
    print(f"   ... and {len(labels) - 10} more")

## Step 4: Fetch Recent Emails

Now let's fetch emails from the last 7 days using our EmailFetcher class.

In [ ]:
# Cell 6: Import email fetcher
from email_agent.email_fetcher import EmailFetcher

# Create fetcher with our Gmail service
email_fetcher = EmailFetcher(gmail_service=gmail_service)

print("✓ Email fetcher initialized")

In [ ]:
# Cell 7: Fetch recent unread messages
print("Fetching unread emails from last 7 days...\n")

# Fetch unread messages
recent_messages = email_fetcher.fetch_unread_messages(
    days_back=7,
    max_results=10  # Start with just 10 for testing
)

print(f"Found {len(recent_messages)} unread messages")
print("\nMessage IDs:")
for i, msg in enumerate(recent_messages, 1):
    print(f"  {i}. {msg['id']}")

## Step 5: Parse Email Content

Now let's parse these emails into a readable format using our EmailParser.

In [ ]:
# Cell 8: Import email parser
from email_agent.email_parser import EmailParser

# Create parser
email_parser = EmailParser()

print("✓ Email parser initialized")

In [ ]:
# Cell 9: Parse first email to see the structure
if recent_messages:
    print("Parsing first email...\n")
    
    # Get full message details
    first_msg_id = recent_messages[0]['id']
    full_message = gmail_service.users().messages().get(
        userId='me',
        id=first_msg_id
    ).execute()
    
    # Parse it
    parsed_email = email_parser.parse_message(full_message)
    
    print("📨 Parsed Email:")
    print(f"\nFrom: {parsed_email['sender']}")
    print(f"Subject: {parsed_email['subject']}")
    print(f"Date: {parsed_email['date']}")
    print(f"\nSnippet: {parsed_email['snippet']}")
    print(f"\nBody preview (first 500 chars):")
    print(f"{parsed_email['body'][:500]}...")
    
    # Calculate metadata
    metadata = email_parser.calculate_metadata(parsed_email)
    print(f"\n📊 Metadata:")
    print(f"   Word count: {metadata['word_count']}")
    print(f"   Character count: {metadata['char_count']}")
    print(f"   URLs found: {metadata['url_count']}")
    print(f"   Est. reading time: {metadata['estimated_read_time_minutes']} min")
else:
    print("No messages to parse!")

## Step 6: Process Multiple Emails

Let's process all the emails we fetched and store them for analysis.

In [ ]:
# Cell 10: Process all fetched emails
print(f"Processing {len(recent_messages)} emails...\n")

parsed_emails = []

for i, msg in enumerate(recent_messages, 1):
    print(f"Processing email {i}/{len(recent_messages)}...", end=" ")
    
    try:
        # Fetch full message
        full_message = gmail_service.users().messages().get(
            userId='me',
            id=msg['id']
        ).execute()
        
        # Parse it
        parsed = email_parser.parse_message(full_message)
        parsed_emails.append(parsed)
        
        print(f"✓ {parsed['subject'][:50]}...")
        
    except Exception as e:
        print(f"✗ Error: {e}")

print(f"\n✅ Successfully parsed {len(parsed_emails)} emails!")

In [ ]:
# Cell 11: Show summary of all emails
print("📧 Email Summary:\n")
print(f"{'#':<4} {'From':<30} {'Subject':<50}")
print("-" * 85)

for i, email in enumerate(parsed_emails, 1):
    # Extract sender name
    sender = email_parser.extract_sender_name(email['sender'])
    sender = sender[:28] + ".." if len(sender) > 30 else sender
    
    # Truncate subject
    subject = email['subject'][:47] + "..." if len(email['subject']) > 50 else email['subject']
    
    print(f"{i:<4} {sender:<30} {subject:<50}")

## Step 7: Save Your Work

Let's save the parsed emails for tomorrow's work.

In [ ]:
# Cell 12: Save checkpoint
import pickle

checkpoint_dir = Path.cwd().parent / 'data' / 'checkpoints'
checkpoint_dir.mkdir(parents=True, exist_ok=True)

checkpoint = {
    'parsed_emails': parsed_emails,
    'timestamp': datetime.now().isoformat(),
    'email_count': len(parsed_emails),
    'date_range_days': 7
}

checkpoint_file = checkpoint_dir / 'day1_2_checkpoint.pkl'
with open(checkpoint_file, 'wb') as f:
    pickle.dump(checkpoint, f)

print(f"✅ Checkpoint saved to: {checkpoint_file}")
print(f"   - Emails processed: {checkpoint['email_count']}")
print(f"   - Timestamp: {checkpoint['timestamp']}")
print(f"\n📝 You can load this later with:")
print(f"   with open('{checkpoint_file}', 'rb') as f:")
print(f"       checkpoint = pickle.load(f)")

## 🎉 Day 1-2 Complete!

**What you accomplished**:
- ✅ Set up Gmail API authentication
- ✅ Successfully connected to your Gmail account
- ✅ Fetched recent unread emails
- ✅ Parsed emails into structured format
- ✅ Saved checkpoint for next session

**Next Steps (Day 3-4)**:
- Create Email Analyzer Agent with autogen
- Analyze each email for key insights
- Extract companies, products, and action items
- Rate importance of each newsletter

**Key Learnings**:
- Gmail API uses OAuth2 for authentication
- Emails can be queried with search operators
- Email bodies are base64 encoded
- Multipart messages need recursive parsing

---

### Time Saved So Far: ~30 minutes
Instead of manually opening 10 emails, you automated the fetching and parsing!